# Laboratory Exercise: Acoustic Sensors

## Milestone Items
The following are required to complete the milestone for this exercise:
- Part 1
    1. Submit to Canvas all three figures you generate using your three lists of values for speed of sound
    2. Submit to Canvas the python code used to create the three figures
- Part 2
    1. Submit to Canvas your figure of the bathymetry along the dock
    2. Submit to Canvas the python code used to create your figure
    
Provide a 2-3 sentence response addressing the questions proposed on the milestone.

# Before we start...

You will need to send the file **hcsr04v2.py** to your microcontroller using mpfshell or a similar tool of your preference. 

# Part 1: Determining the Speed of Sound in Air

For this activity, you will be using a **HCSR04** 40 kHz ultrasonic sensor to determine the speed of sound in air.  This sensor sends out a pulse of sound, then listens for the echo return.  Based on the delay between sending the signal and receiving an echo back, you will be able to determine how far away an object is from the sensor.

In this instance we are unsure of the speed of sound in air.  By collecting measurements at a known distance, we can see how changing the speed of sound affects the accuracy of distance the sensor measures, and narrow in on the correct value.  

## Assembling the HCSR04 Sensor
Begin by connecting your **HCSR04** to your **ESP8266**.  The HCSR04 requires 5 volts, so for this excersize you will use your microcontroller in a breadboard, powered by the USB cable from your computer.  The USB cable provides 5V to the board, which means we can use the *USB* pin on the ESP8266 to give 5V to the HCSR04.

The HCSR04 sensor has 4 pins, *GND*, *VCC*, *Trig*, and *Echo*.  For this excercise, connect the *trig* pin on the HCSR04 to GPIO \#12 on the ESP8266, and the *echo* pin to GPIO \#14.  Connect *GND* to *GND* and *5V* to *USB*.

## Estimating c

We are now going to record a series of measurements at a known distance while adjusting sound speed (`c`) to determine the correct value.  

Let's import the packages we'll need to work through the rest of the lab activity:

In [ ]:
import serial # We will need the serial library to communicate with the sensor via USB
from time import sleep # The 'sleep' function will tell python to wait to hear back from the board
import matplotlib.pyplot as plt # We will use matplotlib to plot the data
#This tells jupyter to show us the plot right in the notebook
%matplotlib inline 

We're using a package called `pyserial` to talk to the microcontroller via USB.  **You will need to set the name of your COM port below.**  If you don't know it, you can check by using BeagleTerm. Edit then run the following cell:

In [ ]:
ser = serial.Serial('YOUR COM PORT NAME HERE',baudrate=115200) # The first argument is the com port of your sensor

Now that we can talk to our sensor through this notebook, we'll start by trying to import the library we need to run the sensor.  The driver for this sensor is the module named `hcsr04v2` that you just put on your microcontroller.  Read through then run the following cell:

In [ ]:
ser.write(b'import hcsr04v2 \r') # send 'import hcsr04' to the board
sleep(1) # wait 1 second before checking for a response
input_data = ser.read(ser.inWaiting()) # read the response
if input_data.find(b'Error') != -1: # If the word 'Error' appears in the response
    print('Import Failed, check serial command or libraries') # print fail message
else: # if no 'Error'
    print('Import Successful') # success!

The `hcsr04v2.HCSR04` class requires three input variables:
- `trigger_pin`,  the GPIO pin on the ESP8266 corresponding to the **Trig** pin on the HCSR04
- `echo_pin`,  the GPIO pin on the ESP8266 corresponding to the **Echo** pin on the HCSR04
- `c`, the speed of sound to use to calculate distance

We're going to pass a command to the board that tells it how to use the acoustic sensor using the following code:

`sensor = hcsr04v2.HCSR04(trigger_pin = 12, echo_pin = 14, c = 300)`

### Step 1 - Define the arguments
We can set up the arguments separately so they're easy to edit when we want to change them:

In [ ]:
triggerPin = 12 #board pin number to 'Trig' on sensor
echoPin = 14 #board pin number to 'Echo' on sensor
soundSpeed = range(300,350,10) # a list of sound speeds, i.e., can be range() or [200]

### Step 2 - Collecting measurements
Now we'll send the setup to the board, and tell it to take a measurement and report back a distance for each value of sound speed we ask.  After defining the sound speed, we will use `sensor.distance()` to take a measurement.

Your ESP8266 should return a float value that corresponds to the distance of the object in front of the sensor in **millimeters**.  Each line from the board will be echoed so we can see the results, and these numbers will get stored in a list.  Read then run the following cell to collect data points for your list of sound speeds:

In [ ]:
distance = [] # empty list to put values in
c = [] # empty list to put values in
for speed in soundSpeed: # for each speed of sound value we previously defined...
    command = 'sensor = hcsr04v2.HCSR04(trigger_pin = '+str(triggerPin)+', echo_pin = '+str(echoPin)+', c = '+str(speed)+')\r'# setup the sensor with that speed of sound
    ser.write(bytes(command, 'utf-8')) # send the command
    ser.write(b'sensor.distance()\r') # take a distance measurement
    sleep(.5) # wait .5 seconds for the microcontroller to respond
    input_data = ser.read(ser.inWaiting()) # read the response
    print('Command Sent: '+ str(input_data).split('\\r\\n>>')[0]) # print out what we sent
    print('Distance measured: ',float(str(input_data).split('\\n')[2][:-2])) # print out what it returned
    distance.append(float(str(input_data).split('\\n')[2][:-2])) # add the distance value to our list
    c.append(float(str(input_data).split('\\n')[0].split('c = ')[-1][:-3])) # add the sound speed value to our list

### Step 3 - Plot the measurements

Fill in the missing information below in order to plot how distance measured changes as a function of sound speed:

In [ ]:
setDistance =  # The known distance
plt.plot(c,distance)
plt.xlabel('c')
plt.ylabel('distance (mm)')
plt.axhline(y=setDistance,linestyle = '--',color = 'g')

Based on the intercept of the dashed line (your known distance) and the solid line (the measured distance at sound speed *c*), create a new list of values for `c` at a finer resolution (intervals of 5?  intervals of 3?) that will help you zoom in on the `c` value of that intercept.

Repeat **Step 1 - Step 3**, each time changing the values used for sound speed, and determine a new set of vales for `distance`. 

Try increasing the resolution of  `c` one more time, trying values with an interval of 1.  Again, define a new set of values for `c` on your ESP8266 and determine a new set of vales for `distance`.  Create a third figure with the data.

If you run the sensor again at this resolution without changing the speed of sound values, do the distances change?  What is the variability, and what might be causing it?

Based on the data and figures, what do you estimate the speed of sound in air to be?  What do you notice about the variability in the distance measurements as you increase the resolution of your `c` list?  What would you do to improve accuracy of your measurement?

Now that you have a more accurate value for the speed of sound, test the limits of your sensor by adjusting the values in **Step 1** and running the code in **Step 2**.  Does there appear to be a minimum distance you can measure?  Does there appear to be a maximum distance?  What happens when it seems like you might be too far away or too close to take a measurement?

# Part 2: Mapping the Seafloor

For this activity, you will be using a **HCSR04** 40 kHz ultrasonic sensor to determine the bathymetry along the MSB dock.  The HCSR04 sensor sends out a pulse of sound through one transducer, and listens for the echo return in the second transducer.  Based on the delay between sending the signal and receiving an echo back, you will be able to determine how far away an object is from the sensor.

This activity uses the waterproof transducer version of the HCSR04 ultrasonic acoustic sensor to create a rough bathymetric chart of a region of off a pier or dock.  For this excercise you will collect measurements of the depth to the seafloor along the Marine Sciences Building dock with 1 meter spacing.

## Bathymetry Measurements
You will be using the HCSR04 to collect measurements of depth at a series of fixed positions along the dock.

### Setting the Sound Speed
First, we need to define the sound speed of the water so we can get a correct measure from distance.  Recall from Part 1 that slight changes in sound speed affect the distance calculated.  We'll define sound speed as the following:


In [ ]:
soundSpeed = [1440] # a list of sound speeds, i.e., can be range() or [200]

### Prepare to Store the Data

Now let's get set up to take some measurements.  We're going to store the measurements in a python dictionary, so let's make an empty one:

In [ ]:
allSamples = {} # let's make an empty dictionary that we'll fill

### Set up the Sensor

Like in Part 1, we're going to have to tell the microcontroller where the acoustic sensor is plugged in, we we can define those two pins below:

In [ ]:
triggerPin = 12 #board pin number to 'Trig' on sensor
echoPin = 14 #board pin number to 'Echo' on sensor

To actually take the measurements, we're going to use `pySerial` like in Part 1.  The following function will be used to send those commands and return a measurement which will be stored in the dictionary we just created. Read through the function below so you understand what is being done, and run the cell so that we can use it. 

In [ ]:
def dist(triggerPin,echoPin,soundSpeed,site,allSamples): #We give the function the pins for the board, soundspeed, measurement site, and our dictionary
    for speed in soundSpeed: # for each sound speed (here, only for the 1 value)
        command = 'sensor = hcsr04v2.HCSR04(trigger_pin = '+str(triggerPin)+', echo_pin = '+str(echoPin)+', c = '+str(speed)+')\r' # set up the sensor on the microcontroller
        ser.write(bytes(command, 'utf-8')) # send the command
        ser.write(b'sensor.distance()\r') # take a distance measurement
        sleep(.5) # wait for .5 seconds for the sensor to respond
        input_data = ser.read(ser.inWaiting()) # read the response
        print('Command Sent: '+ str(input_data).split('\\r\\n>>')[0]) # print out what we sent
        print('Distance measured: ',float(str(input_data).split('\\n')[2][:-2])) # print out the response
        if site in allSamples: # if the site number is already a key in the dictionary...
            allSamples[site].append(float(str(input_data).split('\\n')[2][:-2])+100) # append the new measurement to that site list
        else: # if this is a new site and doesn't have a key yet...
            allSamples[site] = [float(str(input_data).split('\\n')[2][:-2])+100] # make a new key and give it the measurement value
        return allSamples # send us back the updated dictionary

### Collecting Data

Following the tape measure, walk along the dock and take a measurements at the sites spaced 1 meter apart.  Lower the transducer into the water to the tape mark indicating 10cm is at the surface.

Start by setting the site number (which is the same as the meter marker on the tape measure, i.e., 10 meters is site 10):

In [ ]:
site = 1 # Change this value for each site

Next, use the code cell below to collect a sample, and repeat to collect 5 samples at each site.  Do all of the values collected appear to be similar?  Do you think they are correct?

In [ ]:
allSamples = dist(triggerPin,echoPin,soundSpeed,site,allSamples)
print('Measurements for Site '+ str(site), allSamples[site])

We can look at the entire dictionary by running the cell below:

In [ ]:
print(allSamples)

In case you take a measurement by mistake and need to remove it, you can use `allSamples[`*Site Number*`].remove(`*Bad Value*`)` to remove it from the site key.

Once your sample table is complete, create a plot using the cell below.  Add a title and axis labels.

In [ ]:
plt.plot(list(allSamples.keys()),list(allSamples.values()),'.b')
plt.gca().invert_yaxis()

If you have time, try making a new plot using the data in your dictionary that shows the median of the measurements at each distance that you believe to be correct in order to remove noise.  